# Fashion MNIST Image Classification with tf.layers API

This notebook demonstrates how to train a deep neural network model for image classification and deploy it as an Application Programming Interface (API) (i.e. web service) for online predictions.

In [0]:
import os

# do not change these
MODEL_TYPE='dnn'       # deep neural network
os.environ['MODEL_TYPE'] = MODEL_TYPE

def start_tensorboard(logdir, url_file):
  get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(logdir))
  get_ipython().system_raw('lt --port 6006 >> {} 2>&1 &'.format(url_file))
  get_ipython().system('cat {}'.format(url_file))

def stop_tensorboard(url_file):
  get_ipython().system_raw("ps -Af  | grep -E 'tensorboard|lt --port' | awk '{print $2}' | xargs -I % kill -9 %")
  get_ipython().system_raw("rm {}".format(url_file))

In [0]:
%%bash
git clone https://github.com/osipov/training-data-analyst.git
cp -r training-data-analyst/bootcamps/imagereco/fashionmodel .

In [0]:
!find fashionmodel

## Run as a Python module

Now since we want to run our code on Cloud ML Engine, we've packaged it as a Python module.

The `model.py` and `task.py` files containing the model code are in <a href="fashionmodel/trainer">fashionmodel/trainer</a>

In [0]:
%%writefile fashionmodel/trainer/model.py

#!/usr/bin/env python
# Licensed under the Apache License, Version 2.0 See footer for details.

import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

HEIGHT=28
WIDTH=28
NCLASSES=10

def dnn_model(img, mode, hparams):
  X = tf.reshape(img, [-1, HEIGHT*WIDTH]) #flatten input
  h1 = tf.layers.dense(X, 100, activation=tf.nn.relu)
  h2 = tf.layers.dense(h1, 100, activation=tf.nn.relu)
  #TODO: define a tf.layers.dense layer named h3, having h2 layer as input,
  #with 100 neurons in the layer and tf.nn.relu activation function 
  h3 = tf.layers.dense(h2, 100, activation=tf.nn.relu)
  
  ylogits = tf.layers.dense(h3, NCLASSES, activation=None)
  return ylogits, NCLASSES

def train_and_evaluate(output_dir, hparams):
  EVAL_INTERVAL = 60

  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_images = train_images.astype(dtype=np.float32, copy=False).reshape(train_images.shape + (1,))
  test_images = test_images.astype(dtype=np.float32, copy=False).reshape(test_images.shape + (1,))
  train_labels = np.eye(10)[train_labels]
  test_labels = np.eye(10)[test_labels]    
    
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'image': train_images},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True,
    queue_capacity=6000
  )

  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'image':test_images},
    y=test_labels,
    batch_size=100,
    num_epochs=1,
    shuffle=False,
    queue_capacity=6000
  )
  estimator = tf.estimator.Estimator(model_fn = image_classifier,
                                     params = hparams,
                                     config=tf.estimator.RunConfig(
                                         save_checkpoints_secs = EVAL_INTERVAL),
                                     model_dir = output_dir)
  
  train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn,
                                    max_steps = hparams['train_steps'])
  
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  
  eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn,
                                  steps = None,
                                  exporters = exporter,
                                  throttle_secs = EVAL_INTERVAL)
  
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
  
def serving_input_fn():
    #input will be rank 3
    feature_placeholders = {
        'image': tf.placeholder(tf.float32, [None, HEIGHT, WIDTH])}
    
    #but model function requires rank 4
    features = {
        'image': tf.expand_dims(feature_placeholders['image'], -1)} 
    
    return tf.estimator.export.ServingInputReceiver(features, 
                                                    feature_placeholders)

def image_classifier(features, labels, mode, params):
  model_functions = {
      'dnn':dnn_model}
  
  model_function = model_functions[params['model']]  
  
  ylogits, nclasses = model_function(features['image'], mode, params)

  probabilities = tf.nn.softmax(ylogits)
  
  classes = tf.cast(tf.argmax(probabilities, 1), tf.uint8)
  
  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=ylogits, labels=labels))
    
    evalmetrics = {'accuracy': tf.metrics.accuracy(classes, tf.argmax(labels, 1))}
    
    if mode == tf.estimator.ModeKeys.TRAIN:
      # this is needed for batch normalization, but has no effect otherwise
      update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
      with tf.control_dependencies(update_ops):
         train_op = tf.contrib.layers.optimize_loss(
             loss, 
             tf.train.get_global_step(),
             learning_rate=params['learning_rate'], 
             optimizer="Adam")
    else:
      train_op = None
      
  else:
    loss = None
    train_op = None
    evalmetrics = None
 
  return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={"probabilities": probabilities, "classes": classes},
        loss=loss,
        train_op=train_op,
        eval_metric_ops=evalmetrics,
        export_outputs={'classes':tf.estimator.export.PredictOutput(
            {"probabilities": probabilities, "classes": classes})}
    )
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.  

In [0]:
%%bash
rm -rf fashion_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/fashionmodel
python -m trainer.task \
   --output_dir=${PWD}/fashion_trained/${MODEL_TYPE} \
   --train_steps=1000 \
   --learning_rate=0.01 \
   --train_batch_size=143 \
   --model=$MODEL_TYPE

Training should finish in just a few seconds because it ran outside of the Jupyter's runtime, as an independent process of the node's operating system. You can explore the training metrics using TensorBoard.

## Monitoring training with TensorBoard

Use this cell to launch tensorboard

In [0]:
!npm install -g localtunnel
start_tensorboard('./fashion_trained/{}'.format(MODEL_TYPE), 'url')

In [0]:
%sx read -p 'Press Enter in the input box to stop TensorBoard '
stop_tensorboard('url')
print("Stopped")

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>